In [169]:
import pandas as pd
import numpy as np
import requests
import json
import hvplot.pandas
from dotenv import load_dotenv
from datetime import date
import os
from scipy import stats
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
import pad
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
load_dotenv()

True

In [212]:
print(pd.__version__)

1.0.3


In [3]:
fmp_api = os.getenv('FMP_API')

In [ ]:
as_strided()

In [709]:
class TechnicalIndicators:
    
    def __init__(self, ticker):
        
        self.ticker = ticker
    
    def rolling_spearman(self, seqa, seqb, window):
        stridea = seqa.values.strides[0]
        ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
        strideb = seqa.values.strides[0]
        ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
        ar = pd.DataFrame(ssa)
        br = pd.DataFrame(ssb)
        ar = ar.rank(1)
        br = br.rank(1)
        corrs = ar.corrwith(br, 1)
        return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)
    
    def create_price_df(self, period='daily'):
        
        if period =='hourly':
            api_url = 'https://fmpcloud.io/api/v3/historical-chart/1hour'
        else:
            api_url = 'https://fmpcloud.io/api/v3/historical-price-full'
            
        ticker = json.loads(requests.get(f"{api_url}/{self.ticker}?apikey={fmp_api}").content)['historical']
        data = pd.DataFrame(ticker).set_index('date')[::-1]
        data['Date'] = data.index
        data.index = data.index.astype('datetime64[ns]')
        
        return data
        
    def bollinger_bands(self, dataframe,period=20):
        data = dataframe.copy()
        data['middle_band'] = data[['adjClose']].rolling(window=period).mean()
        data[str(period)+'_day_stdev'] = data[['adjClose']].rolling(window=period).std()
        data['upper_band'] = data['middle_band']+2*data[str(period)+'_day_stdev']
        data['lower_band'] = data['middle_band'] - 2*data[str(period)+'_day_stdev']
        data['spread'] = data['upper_band'] + data['lower_band']
        data['change_in_spread'] = data['spread']/data['spread'].shift(1)-1
        data[str(period)+"_return"] = data['adjClose']/data['adjClose'].shift(period)-1
        data['bollinger_signal'] = data['change_in_spread'].rank(ascending=False, pct=True)
        data.dropna()
        
        return data
    
    def dema(self, dataframe, period1=10, period2=20):
        data = dataframe.copy()
        data[str(period1)+'ema1'] = dataframe[['adjClose']].ewm(span=period1, adjust=False).mean()
        data[str(period1)+'ema2'] = data[str(period1)+'ema1'].ewm(span=period1, adjust=False).mean()
        data[str(period1)+'dema'] = 2*data[str(period1)+'ema1'] - data[str(period1)+'ema2']
        data[str(period2)+'ema1'] = data[['adjClose']].ewm(span=period2, adjust=False).mean()
        data[str(period2)+'ema2'] = data[str(period2)+'ema1'].ewm(span=period2, adjust=False).mean()
        data[str(period2)+'dema'] = 2*data[str(period2)+'ema1'] - data[str(period2)+'ema2']
        data[str(period1)+"_return"] = data['adjClose']/data['adjClose'].shift(period1)-1
        data['spread'] = data[str(period1)+'dema'] - data[str(period2)+'dema']
        data['dema_signal'] = data['spread'].rank(ascending=True, pct=True)
        data = data.dropna()
        return data
    
    def price_momentum(self, dataframe, smoothing1=0.0571, smoothing2=0.1, periods1=15, periods2=10):
        data = dataframe.copy()
        data['smoothing_factor'] = smoothing1
        data[str(periods1)+"average"] = data['changeOverTime'].rolling(window=periods1).mean()
        smoothing_factor_list = [data.iloc[periods1][str(periods1)+"average"]]
        data = data.dropna()
        i=1
        j=0
        while i < len(data[str(periods1)+"average"]):
            smoothing_factor = data.iloc[i]['changeOverTime']*data.iloc[i]['smoothing_factor'] + smoothing_factor_list[j]*(1-data.iloc[i]['smoothing_factor'])
            smoothing_factor_list.append(smoothing_factor)
            j+=1
            i+=1
        data['35d_custom_smoothing'] = smoothing_factor_list
        data['35d_custom_10'] = data['35d_custom_smoothing']*10
        data['smoothing_factor2'] = smoothing2
        data[str(periods2)+"average"] = data['35d_custom_10'].rolling(window=periods2).mean()
        data = data.dropna()
        smoothing_factor_list2 = [data.iloc[0][str(periods2)+"average"]]
        i=1
        j=0
        while i < len(data[str(periods2)+"average"]):
            smoothing_factor = (data.iloc[i]['35d_custom_10'] - smoothing_factor_list2[j])*data.iloc[i]['smoothing_factor2'] + smoothing_factor_list2[j]
            smoothing_factor_list2.append(smoothing_factor)
            j+=1
            i+=1
        data[str(periods2)+'d_custom_smoothing'] = smoothing_factor_list2
        data[str(periods2)+"_return"] = data['adjClose']/data['adjClose'].shift(periods2)-1
#         data['signal'] = np.where(data[str(periods2)+'d_custom_smoothing'] > data[str(periods2)+'d_custom_smoothing'].shift(1), 1.0, 0.0)
#         data = data.rename(columns={'signal':'price_mo'})
        return data
    
    def get_ichimoku_cloud(self, dataframe, period1=4, period2=8, period3=15):
        
        #TODO generate signal, ichimoku works better in current market regime with shorter periods, being able to respond faster to events than a traditional version
        # The conversion crossing the base would be the signal
        
        data = dataframe.copy()
        data['conversion_line'] = data[['adjClose']].rolling(window=period1).mean()
        data['base_line'] = data[['adjClose']].rolling(window=period2).mean()
        data['senkou_spanA_line'] = (data['conversion_line']+data['base_line'])/2
        data['senkou_spanB_line'] = data[['adjClose']].rolling(window=period3).mean()
        data['lagging_span'] = data['adjClose'].shift(period2)
        data = data.dropna()
        
        return data
    
    def accumulation_distribution_line(self, dataframe):
        
        ##TODO define periodicity and pass as arguments, use the mean as the signal generator, -1 is buy and and 1 is sell
        ##TODO need to add ability to ignore a -1 during a range of 1s
        
        data = dataframe.copy()
        data['money_flow_mult'] = round(((data['adjClose'] - data['low']) - (data['high'] - data['adjClose']))/(data['high'] - data['low']),2)
        data['money_flow_volume'] = data['money_flow_mult']*data['volume']
        adl = [data.iloc[0]['money_flow_volume']]
        i = 1
        while i < len(data):
            a_d_indicator = adl[i-1]+data.iloc[i]['money_flow_volume']
            adl.append(a_d_indicator)
            i+=1
        data['adl'] = adl
        data['adl_change'] = data['adl']/data['adl'].shift(1)-1
        negative_change_count = [0]*9
        i = 0
        counter = 0
        while i < len(data)-9:
            j=0
            while j < 9:
                if data.iloc[j+i]['adl_change'] <0:
                    counter+=1
                if j %19 == 0:
                    negative_change_count.append(counter)
                    counter = 0
                j+=1
            i+=1
        data['negative_change_counter'] = negative_change_count
        data['9_day_return'] = data['adjClose']/data['adjClose'].shift(9)-1
        data['adl_signal'] = self.rolling_spearman(data['adl'], data['9_day_return'], 9)

        return data
    
    def rsi(self, dataframe, periods=14):
        data = dataframe.copy()
        data['gains'] = np.where(data['changeOverTime']>0, data['changeOverTime'], 0)
        data['losses'] = np.where(data['changeOverTime']<0, np.absolute(data['changeOverTime']), 0)
        data['average_gain'] = data['gains'].rolling(window=periods).mean()
        data['average_loss'] = data['losses'].rolling(window=periods).mean()
        data['rs'] = data['average_gain']/data['average_loss']
        data['rsi'] = (100 - 100/(1+data['rs']))
        data['rsi_signal'] = data['rsi'].rank(ascending=True, pct=True)
        
        return data
    
    
    def get_ratings(self):
        
        ratings = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-rating/{self.ticker}?limit=100&apikey={fmp_api}").content)
        ratings_df = pd.DataFrame(ratings)
        ratings_df['average_rating'] = (ratings_df['ratingScore']+ratings_df['ratingDetailsDCFScore']+ratings_df['ratingDetailsROEScore']+ratings_df['ratingDetailsROAScore'] \
                                        +ratings_df['ratingDetailsDEScore'] + ratings_df['ratingDetailsPEScore']+ ratings_df['ratingDetailsPBScore'])/7
        
        return ratings_df
        
    def get_stock_market_performances(self, dataframe):
        limit = len(dataframe)
        data = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-sectors-performance?limit=525&apikey={fmp_api}").content)
        sector_df = pd.DataFrame(data).set_index('date')
        sector_df_clean = sector_df[::-1]
        s_p500 = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-price-full/^SP500TR?from="+sector_df_clean.index[0]+"&to="+sector_df_clean.index[-1]+"&apikey="+fmp_api).content)['historical']
        sp_df = pd.DataFrame(s_p500)
        sp_df_clean = sp_df[::-1]
        
        return sp_df_clean

    def get_all_indicators(self):
        dataframe = self.create_price_df()
        bb = self.bollinger_bands(dataframe)
        dema = self.dema(dataframe)
        adl = self.accumulation_distribution_line(dataframe)
        rsi = self.rsi(dataframe)
        
        dataframe['bollinger_signal'] = bb['bollinger_signal']
        dataframe['dema_signal'] = dema['dema_signal']
        dataframe['adl_signal'] = adl['adl_signal']
        dataframe['rsi_signal'] = rsi['rsi_signal']
        
        return dataframe


In [710]:
data = pd.read_csv('../FilesExportIndividualStockDFs_Big/A_combined_df.csv')

In [711]:
data.tail()

Date    open    high       low   close  adjClose     volume  \
1464  2021-10-26  159.63  159.88  157.0601  158.19    158.19  1019631.0   
1465  2021-10-27  158.30  158.32  154.8300  155.26    155.26  1485295.0   
1466  2021-10-28  155.98  157.79  154.1900  155.76    155.76  1753116.0   
1467  2021-10-29  155.13  157.57  154.4400  157.49    157.49  2012671.0   
1468  2018-12-05    0.00    0.00    0.0000    0.00      0.00        0.0   

      unadjustedVolume  change  changePercent       vwap  changeOverTime  \
1464         1019631.0   -1.44         -0.902  158.37670        -0.00902   
1465         1485295.0   -3.04         -1.920  156.13667        -0.01920   
1466         1753116.0   -0.22         -0.141  155.91333        -0.00141   
1467         2012671.0    2.36          1.521  156.50000         0.01521   
1468               0.0    0.00          0.000    0.00000         0.00000   

      QUANTITY_FAILS  ShortVolumeNSDQ  ShortExemptVolumeNSDQ  TotalVolumeNSDQ  \
1464           246.0         133949.0                  336.0         260766.0   
1465             0.0          88493.0                 1432.0         227044.0   
1466           216.0         177621.0                    0.0         342292.0   
1467             0.0         111447.0                    0.0         315842.0   
1468            38.0              0.0                    0.0              0.0   

      ShortVolumeNYSE  ShortExemptVolumeNYSE  TotalVolumeNYSE  
1464           8141.0                   33.0          13959.0  
1465           7298.0                  118.0          18013.0  
1466          15864.0                    0.0          25441.0  
1467          12327.0                    0.0          22530.0  
1468              0.0                    0.0              0.0

In [712]:
stock_database = pd.read_csv('../Resources/all_symbol_list.csv').drop('Unnamed: 0', axis=1)

In [713]:
stock_list = list(stock_database['0'])

In [714]:
first_stock_tehcnicals= TechnicalIndicators(stock_list[1])

In [715]:
first_stock_tehcnicals_df = first_stock_tehcnicals.get_all_indicators()

In [716]:
first_stock_tehcnicals_df

open       high        low      close   adjClose      volume  \
date                                                                            
2016-12-05  29.030001  31.277000  28.889999  31.219999  31.151966   7034100.0   
2016-12-06  30.709999  31.190001  30.209999  31.150000  31.082119   3248600.0   
2016-12-07  31.299999  31.889999  30.764999  30.900000  30.832664   5778300.0   
2016-12-08  31.280001  31.840000  30.799999  31.309999  31.241770   3064500.0   
2016-12-09  31.309999  32.110001  31.040001  32.049999  31.980158   4344300.0   
...               ...        ...        ...        ...        ...         ...   
2021-11-26  47.740000  48.140000  45.820000  47.950000  47.950000   6540594.0   
2021-11-29  49.220000  49.935000  47.779000  49.140000  49.140000   6541093.0   
2021-11-30  48.780000  49.380000  45.560000  46.530000  46.530000   7840366.0   
2021-12-01  47.890000  48.045000  43.750000  43.780000  43.780000  15211022.0   
2021-12-02  44.435000  45.290000  43.820100  44.820000  44.820000   5481634.0   

            unadjustedVolume  change  changePercent      vwap  \
date                                                            
2016-12-05         7034100.0   2.190          7.544  30.46233   
2016-12-06         3248600.0   0.440          1.433  30.85000   
2016-12-07         5778300.0  -0.400         -1.278  31.18500   
2016-12-08         3064500.0   0.030          0.096  31.31667   
2016-12-09         4344300.0   0.740          2.363  31.73333   
...                      ...     ...            ...       ...   
2021-11-26         6540594.0   0.210          0.440  47.30333   
2021-11-29         6541093.0  -0.080         -0.163  48.95133   
2021-11-30         7840366.0  -2.250         -4.613  47.15667   
2021-12-01        15211022.0  -4.110         -8.582  45.19167   
2021-12-02         5481634.0   0.385          0.866  44.64337   

                      label  changeOverTime        Date  bollinger_signal  \
date                                                                        
2016-12-05  December 05, 16         0.07544  2016-12-05               NaN   
2016-12-06  December 06, 16         0.01433  2016-12-06               NaN   
2016-12-07  December 07, 16        -0.01278  2016-12-07               NaN   
2016-12-08  December 08, 16         0.00096  2016-12-08               NaN   
2016-12-09  December 09, 16         0.02363  2016-12-09               NaN   
...                     ...             ...         ...               ...   
2021-11-26  November 26, 21         0.00440  2021-11-26          0.417609   
2021-11-29  November 29, 21        -0.00163  2021-11-29          0.328756   
2021-11-30  November 30, 21        -0.04613  2021-11-30          0.516963   
2021-12-01  December 01, 21        -0.08582  2021-12-01          0.712439   
2021-12-02  December 02, 21         0.00866  2021-12-02          0.677706   

            dema_signal  adl_signal  rsi_signal  
date                                             
2016-12-05          NaN         NaN         NaN  
2016-12-06          NaN         NaN         NaN  
2016-12-07          NaN         NaN         NaN  
2016-12-08          NaN         NaN         NaN  
2016-12-09          NaN         NaN         NaN  
...                 ...         ...         ...  
2021-11-26     0.548490   -0.300000    0.634538  
2021-11-29     0.567568   -0.750000    0.653815  
2021-11-30     0.378378   -0.800000    0.584739  
2021-12-01     0.110493   -0.233333    0.342169  
2021-12-02     0.088235    0.066667    0.163855  

[1258 rows x 17 columns]